In [1]:
#pip install unidecode

In [2]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import unidecode
import pandas as pd

Здесь надо ввести текст для поиска вакансии

In [3]:
text= 'копирайтер'

In [4]:
main_link = 'https://hh.ru'
params = {'L_save_area':'true',
          'clusters':'true',
         'enable_snippets':'true',
         'showClusters':'true'}
page = '0'


In [5]:
params['text'] = text
params['page'] = page

In [6]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36'}

In [7]:
response = requests.get(main_link+'/search/vacancy', params=params, headers=headers)

In [8]:
vacancies=[]
while True:
    next_page=None
    if response.ok:
        soup = bs(response.text, 'html.parser')
        next_page = soup.find('a',{'data-qa':"pager-next"})
        vacancies_list = soup.findAll('div', {'class':'vacancy-serp-item'})
        
        
        for vacancy in vacancies_list:
            vacancy_data = {}
            vacancy_all = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'})
            vacancy_link = vacancy_all['href']
            vacancy_name = vacancy_all.getText()
            vacancy_site = "hh.ru"
            vacancy_comp = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
            if (vacancy_comp!=None):
                #обрабатываем строку с зарплатой
                salary_str = vacancy_comp.getText()
                a = salary_str.split(' ')
                currency = a[-1]
                #смотрим - указан ли в зп диапазон
                salary_number = a[-2].split('-')
                #если да - то обрабатываем ситуацию, когда есть и максимальная, и минимальная зп
                if (len(salary_number)==2):
                    salary_min = unidecode.unidecode(salary_number[0])
                    #salary_max = re.sub('\xa',' ',salary_number[1])
                    salary_max = unidecode.unidecode(salary_number[1])
                #если нет - то обрабатываем ситуацию, когда есть только минимальная зп
                else: 
                    if a[0]=='от':
                        salary_min=unidecode.unidecode(salary_number[0])
                        salary_max=None
                        #или когда есть только максимальная зп
                    else:
                        if a[1]=='до':   
                            salary_min=None
                            salary_max=unidecode.unidecode(salary_number[1])

            else: 
                salary_max=None
                salary_min=None
                currency= None
            
            vacancy_data['name'] = vacancy_name
            vacancy_data['link'] = vacancy_link
            vacancy_data['from_site'] = vacancy_site
            vacancy_data['min_compensation'] = salary_min
            vacancy_data['max_compensation'] = salary_max
            vacancy_data['currency'] = currency
            vacancies.append(vacancy_data)
            
        if (next_page!=None):
            next_page_link = main_link+next_page['href']
            response = requests.get(next_page_link, headers=headers)
        else: break

In [9]:
vacancies



[{'name': 'Копирайтер',
  'link': 'https://samara.hh.ru/vacancy/42909347?query=%D0%BA%D0%BE%D0%BF%D0%B8%D1%80%D0%B0%D0%B9%D1%82%D0%B5%D1%80',
  'from_site': 'hh.ru',
  'min_compensation': '50 000',
  'max_compensation': '60 000',
  'currency': 'руб.'},
 {'name': 'Копирайтер',
  'link': 'https://samara.hh.ru/vacancy/42909511?query=%D0%BA%D0%BE%D0%BF%D0%B8%D1%80%D0%B0%D0%B9%D1%82%D0%B5%D1%80',
  'from_site': 'hh.ru',
  'min_compensation': '1 000',
  'max_compensation': None,
  'currency': 'бел.\xa0руб.'},
 {'name': 'Копирайтер',
  'link': 'https://samara.hh.ru/vacancy/42909452?query=%D0%BA%D0%BE%D0%BF%D0%B8%D1%80%D0%B0%D0%B9%D1%82%D0%B5%D1%80',
  'from_site': 'hh.ru',
  'min_compensation': '40 000',
  'max_compensation': '60 000',
  'currency': 'руб.'},
 {'name': 'Проектный менеджер (копирайтер)',
  'link': 'https://samara.hh.ru/vacancy/42970648?query=%D0%BA%D0%BE%D0%BF%D0%B8%D1%80%D0%B0%D0%B9%D1%82%D0%B5%D1%80',
  'from_site': 'hh.ru',
  'min_compensation': None,
  'max_compensation': N

In [10]:
df = pd.DataFrame(vacancies)

In [11]:
df

,name,link,from_site,min_compensation,max_compensation,currency
0,Копирайтер,https://samara.hh.ru/vacancy/42909347?query=%D...,hh.ru,50 000,60 000,руб.
1,Копирайтер,https://samara.hh.ru/vacancy/42909511?query=%D...,hh.ru,1 000,None,бел. руб.
2,Копирайтер,https://samara.hh.ru/vacancy/42909452?query=%D...,hh.ru,40 000,60 000,руб.
3,Проектный менеджер (копирайтер),https://samara.hh.ru/vacancy/42970648?query=%D...,hh.ru,None,None,None
4,Копирайтер,https://samara.hh.ru/vacancy/42909420?query=%D...,hh.ru,40 000,60 000,руб.
...,...,...,...,...,...,...
1995,"Junior специалист E-commerce EN, DE, ES, IT, NL",https://samara.hh.ru/vacancy/42777599?query=%D...,hh.ru,60 000,None,руб.
1996,SMM-менеджер/журналист,https://samara.hh.ru/vacancy/42655070?query=%D...,hh.ru,None,None,None
1997,CRM маркетолог/Digital маркетолог,https://samara.hh.ru/vacancy/42683708?query=%D...,hh.ru,40 000,90 000,руб.
1998,Контент-менеджер,https://samara.hh.ru/vacancy/42643507?query=%D...,hh.ru,None,None,None
